# AMNH CG2 | Class 09 - Annotations | Part 1: Parsing `GFF` files
**Víctor Sojo** | `vsojo@amnh.org`<br/>

Genomes have multiple features within them. Information about these features, such as the location and given names of genes, mRNAs, coding sequences (CDSs), exons, and more, are typically given in **Generic Feature Format (`GFF`)** files.

Here we will parse a `GFF` file containing annotations for the genome of a _Saccharomyces cerevisiae_ strain, as published online by the Yeast Genome consortium.

**References**: Some elements of this lesson are based on chapter 3 from Tiago Antao's [_Bioinformatics with Python Cookbook_](https://www.amazon.com/dp/1789344697/), 2nd Ed. (Packt, 2018), and some others on the [BioPython tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html). [This tutorial](https://www.biostars.org/p/57549/) was also very useful, as was the general [BioPython reference](https://biopython.org/).

## Before we start – Make sure you're using the right `conda` environment
If you're following along the instructions that you must have received separately in a PDF document, you should be here under your `cg2` environment. You can confirm that with:

If you see anything other than `cg2` being printed out, please go back to the PDF and make sure you followed it correctly. You'll have to close this window and then "**Quit**" in the main Jupyter window (if you still have it open, no worries if not). Then make sure you do:<br/>
`jupyter notebook stop`<br/>
`conda activate cg2`<br/>
`jupyter notebook`<br/>

If that still fails, please [send me an email](mailto:vsojo@amnh.org).

## Import libraries & packages
We will need the following:

|Library/Package | Purpose |
|:---------------|:--------|
|`gffutils`      | Allows us to create a database, to be mined |
|`reportlab`     | To graph genomes at the end of this notebook|
|`GenomeDiagram` | As above|
|`Bio.SeqIO`     | To process sequences using Biopython|
|`Image`         | Let's us put images on the notebook|

<br/>
Again, please make sure you followed the instructions in the PDF closely, where the relevant packages were installed.<br/>
Then you can execute the following code by clicking in the cell (i.e. block of text) that contains the code and hitting the play (Run) button above, or hitting Shift+Return on your keyboard.

The above is **Python** code. Most things in a Jupyter Notebook cell are either [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) – such as the text in this cell — or  Python code. However, you can specify other languages too, as we will see in the next cell.

## Download the `GFF` file
We will download the `GFF` file for the genome of **_S. cerevisiae_**. We will download this from the Yeast Genome consortium.

**Technical note:** Whether you use `wget` or `curl -O` for this will depend on whether you're on Linux or Mac OS, so uncomment the appropriate line below by removing the `#` but not the `!`.

This below is one of the many wonders of Jupyter Notebooks: you can run Unix code inside them just as you can run Python code; all you have to do is precede the `bash` code with a `!`.

The file should be downloaded now. We can confirm this by doing `ls -l` and checking that it's really there:

You should see a file called `saccharomyces_cerevisiae.gff.gz` that weighs around `4.6MB`.<br/>
Let's take a look at it, but first we'll need to decompress it. We could use the `!` opener again, but we can actually declare an entire cell as being `bash` code by using the `%%bash` Jupyter "magic".

It's always a good idea to explore the entire file if the size allows, but for now let's just take a look at the end.

This `GFF` file contains a long list of _features_ in the genome of our organism, including what type of feature it is (the 3rd column), which "contig" contains the feature (1st column), where in that contig the feature can be found (4th, 5th and 7th columns), and some hopefully unique information about the feature, such as its ID (last column).

Then, towards the end, it has DNA sequences, which predictably correspond to the contigs (in this case the contigs are the chromosomes, but this will depend on your assembly and type of organism). We can confirm this by trying to find the line that contains the contig for chromosome 1 (plus a couple of lines after it by using `-A2`), which in this annotation is called `chrI`:

OK, that didn't quite work as expected, but it's fine, we got the information we needed.<br/>
(if you want to know the appropriate RegEx, you should do something like `">chrI$"`, which would target only the first chromosome by using the `$` to indicate that the line must end after the `I`)

Let's count the number of **gene**s in the _S. cerevisiae_ genome using `grep`:

...and how many **CDS**s (**C**o**D**ing **S**equences)?

## Creating a `gffutils` database for efficient parsing of the GFF file

Let's create a database (we first remove it if it's already there, so that we're sure we're starting afresh) using `gffutils`:

Note that in the cell above we are combining `bash` code (first line) with `python` code (second line, without the `!`). Cool, innit?

## Parsing the `gffutils` database

### Counting the numbers of features of each kind

Now that we have a `gffutils` database, we can do far more interesting parsing than with `grep`.<br/>
For example, instead of counting each feature one by one, let's count the numbers of all the features in the GFF file:

### Printing all features of the same kind
From the list above, it would seem _S. cerevisiae_ has 17 chromosomes.<br/>
Just to be safe, let's take a look [on Wikipedia](https://en.wikipedia.org/wiki/Saccharomyces_cerevisiae#Genome_sequencing). Here's what I found:
> The _S. cerevisiae_ genome is composed of about 12,156,677 base pairs and 6,275 genes, compactly organized on **16 chromosomes**.

Hm... what's going on here?
Let's print out all of those chromosomes, and their start and end positions (i.e. their lengths)

...ah... mitochondria, nobody remembers poor mitochondria... but try breathing without them!<br/>
Actually, budding yeasts (most strains anyway) can grow anaerobically – which however does not mean they're not using their mitochondria!

### Finding gene counts for each chromosome
We will go over each chromosome and count its number of genes.<br/>
Note that `gffutils` keeps track of which gene belongs to which chromosome, which mRNA belongs to which gene, and so on, so there's a lot we can mine from the database. We will barely scratch the surface here.

## Printing all gene features in a chromosome
The mitochondrial genome has only 28 genes, let's print them all out:

Great. We have the information for all genes in the mitochondrial genome of _S. cerevisiae_. We haven't looked at the sequences themselves yet; let's do that next.

## Creating a BioPython record to contain sequences
We have **features** that we extracted from the GFF, such as gene positions within the genome, and descriptions for those features. However, we haven't used the actual sequences to which these features correspond.

As we saw above, the sequences are elsewhere in the same GFF file, in FASTA format. Let's extract them using BioPython.

To keep things simple and quick, we will work only with one of the chromosomes.

So, for all chromosomes, let's print the first `30` bases of their sequence, and **only for one of the chromosomes** let's extract its sequence and create a BioPython record with it.

Let's take a look at the record we just made for the chosen chromosome:

This record contains the sequence of the entire mitodhondrial chromosome, and no other information beyond the name. Let's add a little more information:

Let's take another look at the record with this new information:

## Adding features to the BioPython `SeqRecord`
You will notice that the `SeqRecord` we just created for the chosen chromosome doesn't contain any `features`; it's pretty much just a sequence for now.

We will add some of those features that we got from the GFF. But, to simulate a slightly more real-world-like scenario, we will do something a little strange: we will extract from the GFF **only** the information about the **positions** of the genes, nothing else.

That is: we are losing all the information about which genes do what, which we had in the original GFF file. This is of course very silly, you wouldn't normally throw away such valuable information. We're only doing it in this example just so that we can then have a starting point in the next lesson.

This looks a little more like the kind of data you would have in your research: you'd have genome sequences, and you may perhaps have some predictions for gene positions, but you wouldn't know _a priori_ what those genes do. We'll try to look into that in the next lesson, and that's why we're getting rid of all the information here.

Let's add the "features" of type `gene` to the BioPython `SeqRecord` for the chosen chromosome of _S. cerevisiae_.

Great! Now we have a sequence record that has both the sequence as such, and where genes (or whatever feature you chose above) are on that sequence.

You will notice that we have just wasted a lot of very useful information, such as the gene ID (which we replaced for an almost meaningless one), what it is predicted to do, and so on.

However, this that we're ending up looks a lot more like the kind of data that you would get in a real project: i.e. you'd have genome sequences, and you may perhaps have some gene predictions, but you wouldn't know a priori what those genes do. We'll try to look into that in another lesson.

## Export all query sequences to FASTA format
Let's export all the sequences each to its own FASTA file.

While we're at it, let's also make a list to contain the gene IDs, so that we can use them later.

Write those gene IDs to a file, so that we can use them later

## Exporting the sequence with features (e.g. gene positions) as a GenBank file
Let's export this new record to a GenBank file, which, similar to a GFF, includes both gene positions and the full sequence.

Why, you may ask, are we exporting as a GenBank file instead of GFF? Well, just so you witness a major bane of bioinformatics: many many types of files that do partly the same, but not quite.

But before we export the file, let's add a bit more information to the record, such as the organism, and where we got the data.

In your real-world case, you would probably add where you sampled this organism, and any useful information you have about it that you think would be good to have in this file. Don't hold back here, more information is better than less.

Now we can export the file to the `genbank` format.

## Visualising (a tiny bit of) the genome
We now have a sequence record, which we exported to `genbank` format. This record is still in memory, so we can keep using it for other purposes. For example, Let's plot the genome of the chosen chromosome of _S. cerevisiae_.

First, we declare the `GenomeDiagram` objects that will hold our genome for plotting.

Next we add each of the genes to the graph, in their appropriate locations.

We alternate colours for even and odd genes, just so that we can see them when they overlap.

And finally, make the plot, using the circular mode.

Here's the image we just created:

**Warning!**<br/>

We're plotting the chromosome as circular. This looks very pretty, but... is it true? Maybe for the mitochondrial chromosome (`chrmt`) it would have been, but for the others?

Like with everything in bioinformatics, you need to be sure that what you're doing makes sense. The computer knows no biology.

On that note, no part of this lesson is meant for biological correctness. There may be plenty of errors (biological or otherwise) in the assumptions above. The intention here was only to demonstrate some of the bioinformatics principles.